
# Assignment - 3  
## Multi-Agent Drone Delivery System using Hybrid Agent Architecture

### Objective
Design a multi-agent drone delivery system using hybrid agent architectures.

### Problem Statement
Design a drone system that:
- Delivers packages in urban environments  
- Avoids obstacles  
- Handles weather disturbances  
- Optimizes energy usage  
- Reschedules deliveries dynamically  
- Coordinates 3 drones  



## 1. System Architecture (Hybrid Agent Design)

Each drone follows a **Hybrid Architecture** consisting of:

1. **Reactive Layer**
   - Immediate obstacle avoidance
   - Wind disturbance correction
   - Collision avoidance with other drones

2. **Deliberative Layer**
   - Path planning using A* algorithm
   - Energy optimization
   - Weather-aware routing

3. **Learning Layer (Optional Extension)**
   - Energy consumption prediction
   - Weather pattern adaptation

### Multi-Agent Coordination
A Central Coordinator Agent:
- Assigns deliveries
- Monitors battery levels
- Reschedules tasks dynamically
- Avoids inter-drone collision


In [1]:

import heapq
import random

GRID_SIZE = 10

# Create urban grid (0 = free, 1 = obstacle)
def create_environment():
    grid = [[0 for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
    
    # Random obstacles
    for _ in range(15):
        x = random.randint(0, GRID_SIZE-1)
        y = random.randint(0, GRID_SIZE-1)
        grid[x][y] = 1
    
    return grid

environment = create_environment()
environment


[[0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 1, 0, 0, 0, 1, 0, 1],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]

In [2]:

def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def astar(grid, start, goal):
    rows, cols = len(grid), len(grid[0])
    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = {start: 0}
    
    while open_set:
        _, current = heapq.heappop(open_set)
        
        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]
        
        neighbors = [(0,1),(1,0),(0,-1),(-1,0)]
        for dx, dy in neighbors:
            neighbor = (current[0]+dx, current[1]+dy)
            
            if 0 <= neighbor[0] < rows and 0 <= neighbor[1] < cols:
                if grid[neighbor[0]][neighbor[1]] == 1:
                    continue
                
                tentative_g = g_score[current] + 1
                if neighbor not in g_score or tentative_g < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g
                    f_score = tentative_g + heuristic(neighbor, goal)
                    heapq.heappush(open_set, (f_score, neighbor))
    
    return None


In [3]:

class Drone:
    def __init__(self, drone_id, position):
        self.id = drone_id
        self.position = position
        self.battery = 100
        self.task = None
        
    def assign_task(self, delivery_point):
        self.task = delivery_point
        
    def plan_route(self, grid):
        if self.task:
            return astar(grid, self.position, self.task)
        return None
    
    def move(self, new_position):
        self.position = new_position
        self.battery -= 1
        
    def handle_weather(self):
        # Simulate wind disturbance
        wind = random.choice([0, 1])  # 1 means disturbance
        if wind:
            self.battery -= 2


In [4]:

class Coordinator:
    def __init__(self, drones):
        self.drones = drones
        
    def assign_deliveries(self, deliveries):
        for drone, delivery in zip(self.drones, deliveries):
            drone.assign_task(delivery)
    
    def reschedule(self):
        for drone in self.drones:
            if drone.battery < 20:
                print(f"Drone {drone.id} low battery! Rescheduling...")
                drone.task = None


In [5]:

# Initialize drones
drones = [
    Drone(1, (0,0)),
    Drone(2, (9,0)),
    Drone(3, (0,9))
]

coordinator = Coordinator(drones)

deliveries = [(8,8), (5,5), (9,9)]
coordinator.assign_deliveries(deliveries)

# Simulate
for drone in drones:
    path = drone.plan_route(environment)
    if path:
        for step in path[1:]:
            drone.handle_weather()
            drone.move(step)
    print(f"Drone {drone.id} final position: {drone.position}, Battery: {drone.battery}")


Drone 1 final position: (8, 8), Battery: 66
Drone 2 final position: (5, 5), Battery: 83
Drone 3 final position: (9, 9), Battery: 81
